In [1]:
import polars as pl

In [22]:
q = (
    pl.scan_csv('data/domestic_cert_ca_tbl.csv',
                 infer_schema_length=0) #all as strings
    .select(pl.col(['LMK_KEY',
                    'ADDRESS1',
                    'ADDRESS2',
                    'ADDRESS3',
                    'POSTCODE',
                    'CURRENT_ENERGY_RATING',
                    'LOCAL_AUTHORITY',
                    'PROPERTY_TYPE',
                    'LODGEMENT_DATE',
                    'TRANSACTION_TYPE',
                    'ENVIRONMENT_IMPACT_CURRENT',
                    'TENURE',
                    'UPRN']))
    .with_columns([pl.col('LODGEMENT_DATE').str.to_date('%Y-%m-%d'),
                  pl.col('ENVIRONMENT_IMPACT_CURRENT').cast(pl.Int64),
                  pl.col('UPRN').cast(pl.Int64)])
    .sort(pl.col(['UPRN', 'LODGEMENT_DATE']))
    .groupby('UPRN').last()
)

In [23]:
%%time
epc_latest_certs = q.collect()

CPU times: total: 1min 36s
Wall time: 35 s


In [24]:
epc_latest_certs.glimpse()

Rows: 4659361
Columns: 13
$ UPRN                        <i64> None, 30, 97, 100, 103, 104, 105, 106, 107, 109
$ LMK_KEY                     <str> 7dcb07a80200d9cfde763f0fae99f1d329abb67a7c0a438c2abe86f115cd03c1, 945145029922013062317101139638477, 1647265119642018071017301852989308, c59230c17d9b975cd1ba5457629112006a624d024894858dd75c712939ace92f, 248960a413bfe4ea746cd67d64f65b144b2343128715131a5e508aa02234fada, 883728034532015041312250532978807, 1240896913412015041413072693950436, 1679813059832018111912231061978999, 1542211424712017050910121296030754, 1280502619602015021114473434359198
$ ADDRESS1                    <str> 103 Grayling Way, Birchington, 9, Charlton Gardens, 13 CHARLTON GARDENS, 15 Charlton Gardens, 6, Charlton Gardens, 17, Charlton Gardens, 14, Charlton Gardens, 19, Charlton Gardens, 21, Charlton Gardens
$ ADDRESS2                    <str> Ryton Central, Severn Road, None, None, None, None, None, None, None, None
$ ADDRESS3                    <str> None, Hallen, None, No

In [25]:
epc_latest_certs.write_csv('data/epc_subset_polars_last.csv')